In [73]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
import random
from tensorflow import keras
import numpy as np
# from tensorflow.keras import layers, Dense, Input, InputLayer, Flatten
# from tensorflow.keras.models import Sequential, Model


from keras.models import Sequential
from keras import layers
from keras.layers import Dense,Input, InputLayer, Flatten


from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline



from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
# from keras.optimizers import  adam
from tensorflow.keras.optimizers import SGD,RMSprop, Adam

from keras.utils import np_utils
from sklearn.tree import DecisionTreeClassifier 
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import random 
from numpy import *
from PIL import Image
import theano
import seaborn as sns

In [4]:
path_test = "./Datasets/training-data/"

In [6]:
CATEGORIES = ['bright_dune', 'crater','dark_dune','impact_ejecta','slope_streak']

In [7]:
training = []

def createTrainingData():
  for category in CATEGORIES:
    path = os.path.join(path_test, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      img_array = cv2.imread(os.path.join(path,img))
      new_array = cv2.resize(img_array, (227, 227))
      training.append([new_array, class_num])
      
createTrainingData()

In [8]:
random.shuffle(training)
X = []
y = []

for features, label in training:
  X.append(features)
  y.append(label)
X = np.array(X).reshape(-1, 227, 227, 3)
y = np.array(y)

In [9]:
X = X.astype('float32')
X /= 255

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4)

In [2]:
model = keras.models.load_model(r'./models/cnn2.h5')

In [26]:
Y_pred = model.predict(X_test)

In [64]:
y_pred = []

In [65]:
for i in range(len(Y_pred)):
    y_pred.append(np.argmax(Y_pred[i]))

AUC-ROC

In [83]:
# Importing all necessary libraries
from sklearn.metrics import roc_curve, auc

class_probabilities = model.predict_proba(X_test)
preds = class_probabilities[:, 1]

fpr, tpr, threshold = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

# Printing AUC
print(f"AUC for our classifier is: {roc_auc}")

# Plotting the ROC
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

ValueError: multiclass format is not supported

In [76]:
from sklearn.metrics import classification_report

In [79]:
sk_report = classification_report(
    digits=6, #floating point accuracy
    y_true=y_test, 
    y_pred=y_pred)
print(sk_report)

              precision    recall  f1-score   support

           0   0.913349  0.975000  0.943168       400
           1   0.958005  0.939914  0.948873      1165
           2   0.962963  0.859504  0.908297       242
           3   0.933333  0.823529  0.875000       102
           4   0.872047  0.932632  0.901322       475

    accuracy                       0.931208      2384
   macro avg   0.927940  0.906116  0.915332      2384
weighted avg   0.932834  0.931208  0.931162      2384



In [80]:
from sklearn.metrics import roc_auc_score

def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):

  #creating a set of all the unique classes using the actual class list
  unique_class = set(actual_class)
  roc_auc_dict = {}
  for per_class in unique_class:
    #creating a list of all the classes except the current class 
    other_class = [x for x in unique_class if x != per_class]

    #marking the current class as 1 and all other classes as 0
    new_actual_class = [0 if x in other_class else 1 for x in actual_class]
    new_pred_class = [0 if x in other_class else 1 for x in pred_class]

    #using the sklearn metrics method to calculate the roc_auc_score
    roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
    roc_auc_dict[per_class] = roc_auc

  return roc_auc_dict

In [81]:
roc_auc_dict = roc_auc_score_multiclass(y_test, y_pred)
roc_auc_dict

{0: 0.9781754032258064,
 1: 0.9502688124720537,
 2: 0.927884652483583,
 3: 0.9104500695983915,
 4: 0.9492911692536737}

Confusion Matrix


In [69]:
plt.figure(figsize = (18,8))
sns.heatmap(metrics.confusion_matrix(y_test, y_pred), annot = True, xticklabels = y_test.unique(), yticklabels = y_test.unique(), cmap = 'summer')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

NameError: name 'sns' is not defined

<Figure size 1800x800 with 0 Axes>